# Setup for Conservation-Based Synthetic Lethal Discovery Pipeline

This notebook outlines our approach to pre-processing publicly-available datasets to facillitate the discovery of synthetic lethal interactions conserved between human and yeast speicies. The wrangled data are uploaded to bigquery to facillitate .sql-based querying (see pipline notebook conservedSL.ipynb).

In [89]:
# load required modules
import pandas as pd

### Reference Dataset for Ortholog Mapping

Our source for human to yeast ortholog mapping is the [Alliance of Genome Resources (AGR)](https://www.alliancegenome.org) - Release 3.0.1. 
Full download is found by navigating from the home page to Data -> Downloads -> Orthology. We are using the 'Alliance combined orthology data' found on the [downloads](https://www.alliancegenome.org/downloads#orthology) page. 

#### Option 1: pre-download combined orthology data

In [90]:
# local file path to download location
# AGR_ORTHOLOGS = 'data/sgd_alliance/ORTHOLOGY-ALLIANCE_COMBINED_4.tsv' 

#### Option 2: web-based download of combined orthology data

In [91]:
# url to file download
AGR_ORTHOLOGS = 'http://download.alliancegenome.org/3.0.1/ORTHOLOGY-ALLIANCE/COMBINED/ORTHOLOGY-ALLIANCE_COMBINED_4.tsv'

In [92]:
ortholog_table = pd.read_csv(AGR_ORTHOLOGS, sep='\t', comment='#')
ortholog_table.head()

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,HGNC:28697,ZADH2,NCBITaxon:9606,Homo sapiens,SGD:S000004937,ADH6,NCBITaxon:559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes
1,FB:FBgn0033188,Drat,NCBITaxon:7227,Drosophila melanogaster,SGD:S000004937,ADH6,NCBITaxon:559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes
2,MGI:2448516,Cryzl2,NCBITaxon:10090,Mus musculus,SGD:S000004937,ADH6,NCBITaxon:559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes
3,WB:WBGene00014096,ZK829.7,NCBITaxon:6239,Caenorhabditis elegans,SGD:S000004937,ADH6,NCBITaxon:559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes
4,RGD:1304982,Tp53i3,NCBITaxon:10116,Rattus norvegicus,SGD:S000004937,ADH6,NCBITaxon:559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,9,Yes,Yes


#### Data Cleaning

In [93]:
# data cleaning - remove identifier names within each field
# e..g HGNC:28697 --> 28697
headers = ortholog_table.columns.values
cols_to_clean = headers[['ID' in s for s in headers]]

In [94]:
def remove_column_annotation(gene_info):
    """ Clean up identifier columns
    Remove the exact source info and keep true identifiers 
    e.g. remove 'MGI:' and keep mouse gene identifier
    """
    return(gene_info.str.split(':').str[-1])

In [95]:
df = ortholog_table.apply(lambda x: remove_column_annotation(x) if x.name in cols_to_clean else x)
df

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,28697,ZADH2,9606,Homo sapiens,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes
1,FBgn0033188,Drat,7227,Drosophila melanogaster,S000004937,ADH6,559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes
2,2448516,Cryzl2,10090,Mus musculus,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes
3,WBGene00014096,ZK829.7,6239,Caenorhabditis elegans,S000004937,ADH6,559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes
4,1304982,Tp53i3,10116,Rattus norvegicus,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,9,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560951,12340,TRPS1,9606,Homo sapiens,1927616,Trps1,10090,Mus musculus,PANTHER|HGNC|PhylomeDB|OrthoFinder|InParanoid|...,9,11,Yes,Yes
560952,ZDB-GENE-030131-8404,trps1,7955,Danio rerio,1927616,Trps1,10090,Mus musculus,ZFIN|PANTHER|PhylomeDB|OrthoFinder|InParanoid|...,7,11,Yes,Yes
560953,WBGene00017033,nekl-4,6239,Caenorhabditis elegans,2685128,Nek10,10090,Mus musculus,PANTHER|PhylomeDB|OrthoFinder|InParanoid|Ortho...,8,10,Yes,Yes
560954,18592,NEK10,9606,Homo sapiens,2685128,Nek10,10090,Mus musculus,PANTHER|HGNC|Ensembl Compara|OMA|PhylomeDB|Ort...,11,11,Yes,Yes


In [96]:
# add column: Algorithm Match Percentage - the % of algorithms that agree on the ortholog mapping 
df["AlgorithmsMatchPerc"] = df["AlgorithmsMatch"]/df["OutOfAlgorithms"] 
df

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore,AlgorithmsMatchPerc
0,28697,ZADH2,9606,Homo sapiens,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes,0.200000
1,FBgn0033188,Drat,7227,Drosophila melanogaster,S000004937,ADH6,559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes,0.200000
2,2448516,Cryzl2,10090,Mus musculus,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,10,Yes,Yes,0.200000
3,WBGene00014096,ZK829.7,6239,Caenorhabditis elegans,S000004937,ADH6,559292,Saccharomyces cerevisiae,PANTHER|Ensembl Compara,2,10,Yes,Yes,0.200000
4,1304982,Tp53i3,10116,Rattus norvegicus,S000004937,ADH6,559292,Saccharomyces cerevisiae,OMA|Ensembl Compara,2,9,Yes,Yes,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560951,12340,TRPS1,9606,Homo sapiens,1927616,Trps1,10090,Mus musculus,PANTHER|HGNC|PhylomeDB|OrthoFinder|InParanoid|...,9,11,Yes,Yes,0.818182
560952,ZDB-GENE-030131-8404,trps1,7955,Danio rerio,1927616,Trps1,10090,Mus musculus,ZFIN|PANTHER|PhylomeDB|OrthoFinder|InParanoid|...,7,11,Yes,Yes,0.636364
560953,WBGene00017033,nekl-4,6239,Caenorhabditis elegans,2685128,Nek10,10090,Mus musculus,PANTHER|PhylomeDB|OrthoFinder|InParanoid|Ortho...,8,10,Yes,Yes,0.800000
560954,18592,NEK10,9606,Homo sapiens,2685128,Nek10,10090,Mus musculus,PANTHER|HGNC|Ensembl Compara|OMA|PhylomeDB|Ort...,11,11,Yes,Yes,1.000000


### Data Filtering
Keep ortholog mapping information for relevant species for our purposes

In [97]:
# create human to yeast mapping table
human2yeast = df.loc[(df['Gene1SpeciesName'] == 'Homo sapiens') & (df['Gene2SpeciesName'] == 'Saccharomyces cerevisiae')]
human2yeast = human2yeast.rename(columns={'Gene1ID': 'HumanID', 'Gene1Symbol': 'HumanSymbol',
                                'Gene2ID': 'YeastID', 'Gene2Symbol': 'YeastSymbol',})

human2yeast = human2yeast.filter(items=['HumanID', 'HumanSymbol', 'YeastID', 'YeastSymbol',
                'Algorithms', 'AlgorithmsMatch', 'OutOfAlgorithms', 'AlgorithmsMatchPerc',
                          'IsBestScore', 'IsBestRevScore',])
human2yeast.head()

,HumanID,HumanSymbol,YeastID,YeastSymbol,Algorithms,AlgorithmsMatch,OutOfAlgorithms,AlgorithmsMatchPerc,IsBestScore,IsBestRevScore
0,28697,ZADH2,S000004937,ADH6,OMA|Ensembl Compara,2,10,0.2,Yes,Yes
5,33848,COA5,S000003795,PET191,PANTHER|PhylomeDB|OrthoFinder|OrthoInspector|H...,7,10,0.7,Yes,Yes
11,20154,ABHD4,S000003342,CLD1,PANTHER|Ensembl Compara|PhylomeDB|Hieranoid|Tr...,6,10,0.6,No,Yes
12,21396,ABHD5,S000003342,CLD1,PANTHER|Ensembl Compara|PhylomeDB|Hieranoid|Tr...,6,10,0.6,No,Yes
19,10524,SALL1,S000006217,CMR3,PANTHER|Roundup,2,10,0.2,Yes,Yes


In [98]:
human2yeast.infer_objects()
human2yeast['HumanID']=human2yeast['HumanID'].astype(str).astype(int)

In [99]:
human2yeast.dtypes

HumanID                  int64
HumanSymbol             object
YeastID                 object
YeastSymbol             object
Algorithms              object
AlgorithmsMatch          int64
OutOfAlgorithms          int64
AlgorithmsMatchPerc    float64
IsBestScore             object
IsBestRevScore          object
dtype: object

In [100]:
# create yeast to human mapping table
yeast2human = df.loc[(df['Gene1SpeciesName'] == 'Saccharomyces cerevisiae') & (df['Gene2SpeciesName'] == 'Homo sapiens')]
yeast2human = yeast2human.rename(columns={'Gene1ID': 'YeastID', 'Gene1Symbol': 'HumanSymbol',
                                'Gene2ID': 'HumanID', 'Gene2Symbol': 'YeastSymbol',})
yeast2human = yeast2human.filter(items= ['YeastID', 'YeastSymbol','HumanID', 'HumanSymbol',
                         'Algorithms', 'AlgorithmsMatch', 'OutOfAlgorithms', 'AlgorithmsMatchPerc',
                            'IsBestScore', 'IsBestRevScore'])
yeast2human.head()

,YeastID,YeastSymbol,HumanID,HumanSymbol,Algorithms,AlgorithmsMatch,OutOfAlgorithms,AlgorithmsMatchPerc,IsBestScore,IsBestRevScore
286719,S000002326,TEX13A,11735,NRP1,OrthoFinder|Ensembl Compara,2,10,0.2,Yes,Yes
286733,S000000321,TKT,11834,TKL2,Ensembl Compara|OMA|PhylomeDB|OrthoFinder|InPa...,9,10,0.9,Yes,Yes
286734,S000006278,TKT,11834,TKL1,Ensembl Compara|OMA|PhylomeDB|OrthoFinder|InPa...,9,10,0.9,Yes,Yes
286750,S000001295,PRKAR1B,9390,BCY1,PANTHER|OMA|OrthoFinder|OrthoInspector|TreeFam...,6,10,0.6,No,Yes
286756,S000005041,ING2,6063,PHO23,PANTHER|Ensembl Compara|OrthoInspector|Hierano...,6,10,0.6,Yes,Yes


In [101]:
yeast2human['HumanID']=yeast2human['HumanID'].astype(str).astype(int)

In [102]:
# write ortholog mapping tables to file
yeast2human.to_csv(path_or_buf='data/sgd_alliance/yeast2human_alliance_v4.csv', index=False)
human2yeast.to_csv(path_or_buf='data/sgd_alliance/human2yeast_alliance_v4.csv', index=False)

### Create BigQuery Dataset and Upload Data


In [103]:
# google cloud authentication
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=uNVhind8gJVqxcyEgUv8OKO9u4ooK2XlqrKVqbO2hfw&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth



Credentials saved to file: [/Users/nchamb/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "syntheticlethality" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [104]:
# pip install pyarrow
from google.cloud import bigquery

In [105]:
# configure project info and bigquery client
project='syntheticlethality'

# construct a BigQuery client object.
client = bigquery.Client(project) # Replace XXXXXXXX with your project ID

/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


#### Create a bigquery dataset within your project 


In [106]:
# if it does not already exist
dataset_name='Orthology'
dataset_id = bigquery.Dataset(project +'.'+ dataset_name)
try:
    client.get_dataset(dataset_id)
    print("Dataset: {} already exists".format(project, dataset_name))
except:
    # construct a full Dataset object to send to the API.
    dataset = bigquery.Dataset(dataset_id)
    
    # send the dataset to the API for creation.
    dataset = client.create_dataset(dataset_id)  # Make an API request.
    print("Created dataset: {}.{}".format(project_id,  dataset_id))

Dataset: syntheticlethality already exists


#### Configure upload job - Yeast2Human Table


In [107]:
yeast2human.columns.values

array(['YeastID', 'YeastSymbol', 'HumanID', 'HumanSymbol', 'Algorithms',
       'AlgorithmsMatch', 'OutOfAlgorithms', 'AlgorithmsMatchPerc',
       'IsBestScore', 'IsBestRevScore'], dtype=object)

In [108]:
print(yeast2human.dtypes) 

YeastID                 object
YeastSymbol             object
HumanID                  int64
HumanSymbol             object
Algorithms              object
AlgorithmsMatch          int64
OutOfAlgorithms          int64
AlgorithmsMatchPerc    float64
IsBestScore             object
IsBestRevScore          object
dtype: object


In [109]:
table_description = '''
Mapping conserved genes from yeast (Saccharomyces cerevisiae) to human (Homo sapiens) derived from Integrated orthology inferences created using the Drosophila RNAi Screening Center(DRSC) Integrative Ortholog Prediction Tool (DIOPT) 
at Harvard Medical School see more information at: https://www.flyrnai.org/diopt. 

Table downloaded from the Alliance for Genome Resources. Release 3.0.1"
https://www.alliancegenome.org/downloads#orthology
'''

In [110]:
job_config = bigquery.LoadJobConfig(
   destination_table_description=table_description,
    
    # specify table schema
    schema=[
        bigquery.SchemaField(name="YeastID", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Saccharomyces Genome Database (SGD) gene identifier'),
        bigquery.SchemaField(name="YeastSymbol", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description="Yeast official gene symbol"),
        bigquery.SchemaField(name="HumanID", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='HGNC gene identifier'),
        bigquery.SchemaField(name="HumanSymbol", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='HGNC gene symbol'),
       bigquery.SchemaField(name="Algorithms", field_type=bigquery.enums.SqlTypeNames.STRING, 
                          description='The orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="AlgorithmsMatch", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='The number of orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="OutOfAlgorithms", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='The toal number of orthology methods that could make the orthology association for the species'),
        bigquery.SchemaField(name="AlgorithmsMatchPerc", field_type=bigquery.enums.SqlTypeNames.FLOAT, 
                             description='The proportion of orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="IsBestScore", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Within the species, whether this gene is called the ortholog of the input gene by the highest number of algorithms'),
        bigquery.SchemaField(name="IsBestRevScore", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Within the species of the input gene, whether the input gene is called the ortholog of the gene by the highest number of algorithms'), 
    ],
    
    write_disposition="WRITE_TRUNCATE" #replaces the table with the loaded data
)


In [111]:
# create table name
table_name = 'YEAST2HUMAN'
table_id = "{}.{}.{}".format(project,  dataset_name, table_name)

#### Load tables to bigquery

In [112]:
job = client.load_table_from_dataframe(yeast2human, table_id, job_config=job_config)

In [113]:
job.result()  # Wait for the job to complete.


In [114]:
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 6124 rows and 10 columns to syntheticlethality.Orthology.YEAST2HUMAN


#### Configure upload job - Human2Yeast Table

In [115]:
table_description = '''
Mapping conserved genes from human (Homo sapiens) to yeast (Saccharomyces cerevisiae) to  derived from Integrated orthology inferences created using the Drosophila RNAi Screening Center(DRSC) Integrative Ortholog Prediction Tool (DIOPT) 
at Harvard Medical School see more information at: https://www.flyrnai.org/diopt. 

Table downloaded from the Alliance for Genome Resources. Release 3.0.1"
https://www.alliancegenome.org/downloads#orthology
'''

In [116]:
human2yeast.columns.values

array(['HumanID', 'HumanSymbol', 'YeastID', 'YeastSymbol', 'Algorithms',
       'AlgorithmsMatch', 'OutOfAlgorithms', 'AlgorithmsMatchPerc',
       'IsBestScore', 'IsBestRevScore'], dtype=object)

In [117]:
# configure upload
job_config = bigquery.LoadJobConfig(
   destination_table_description=table_description,
    
    # specify table schema
    schema=[
        bigquery.SchemaField(name="HumanID", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='HGNC gene identifier'),
        bigquery.SchemaField(name="HumanSymbol", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='HGNC gene symbol'),
               bigquery.SchemaField(name="YeastID", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Saccharomyces Genome Database (SGD) gene identifier'),
        bigquery.SchemaField(name="YeastSymbol", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description="Yeast official gene symbol"),
       bigquery.SchemaField(name="Algorithms", field_type=bigquery.enums.SqlTypeNames.STRING, 
                          description='The orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="AlgorithmsMatch", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='The number of orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="OutOfAlgorithms", field_type=bigquery.enums.SqlTypeNames.INTEGER, 
                             description='The toal number of orthology methods that could make the orthology association for the species'),
        bigquery.SchemaField(name="AlgorithmsMatchPerc", field_type=bigquery.enums.SqlTypeNames.FLOAT, 
                             description='The proportion of orthology methods that make the orthology association for the species'),
        bigquery.SchemaField(name="IsBestScore", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Within the species, whether this gene is called the ortholog of the input gene by the highest number of algorithms'),
        bigquery.SchemaField(name="IsBestRevScore", field_type=bigquery.enums.SqlTypeNames.STRING, 
                             description='Within the species of the input gene, whether the input gene is called the ortholog of the gene by the highest number of algorithms'), 
    ],
    
    write_disposition="WRITE_TRUNCATE" #replaces the table with the loaded data
)


In [118]:
# create table name
table_name = 'HUMAN2YEAST'
table_id = "{}.{}.{}".format(project,  dataset_name, table_name)

In [119]:
job = client.load_table_from_dataframe(human2yeast, table_id, job_config=job_config)

In [120]:
job.result()

In [121]:
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 6124 rows and 10 columns to syntheticlethality.Orthology.HUMAN2YEAST
